In [1]:
# generic libraries
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize

# project specific custom libraries
from config import *
from simulation import simulate_decay_times
from analysis import binned_maximum_likelihood_fit

In [2]:
#set a seed for numpys random number generator
np.random.seed(42)

In [ ]:
# Exercise part 3 Simple simulation

# Part 0, generate the plot from the N function
t_bounds = [0, 2e-5]
n = 10000

t_space = np.linspace(t_bounds[0], t_bounds[1], n)
t_computed_values = N(t_space, n)

# Part 1, simulate n decay times and create a histogram plot
simulated_decay_points = simulate_decay_times(n, t_bounds[1])

# create the histogram
counts, bins = np.histogram(simulated_decay_points[:, 0] , bins=35)
plt.hist(bins[:-1], bins, weights=counts, color='lightgray', edgecolor='black')
plt.xlabel('Time [s]')
plt.ylabel('Counts')
plt.title(f'Histogram of {n} simulated decay times')
plt.savefig('../Documentation/images/simulated_decay_histogram.png', dpi=750)
plt.close()

muon_estimate, pion_estimate, muon_uncertainty, pion_uncertainty = binned_maximum_likelihood_fit(counts, bins)
print("Muon estimate: ", muon_estimate)
print("Muon uncertainty: ", muon_uncertainty)
print("Pion estimate: ", pion_estimate) 
print("Pion uncertainty: ", pion_uncertainty)


/opt/homebrew/lib/python3.11/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


1.7603709619112948e-08


TypeError: cannot unpack non-iterable NoneType object